In [11]:
!pip install dspy-ai

Defaulting to user installation because normal site-packages is not writeable


In [12]:
# 导入包
import json
import dspy
import ast

In [13]:
# 设置模型

lm = dspy.LM('ollama/qwen2.5:3b', api_base="http://192.168.1.131:11434")
dspy.configure(lm=lm)

In [14]:
# 创建签名类

# class InitializeInventory(dspy.Signature):
#     """初始化库存的签名"""
#     inventory_info = dspy.OutputField(desc="商品和库存信息")

class SelectProduct(dspy.Signature):
    """选择商品的签名"""
    available_products = dspy.InputField(desc="可用商品列表")
    product_number = dspy.InputField(desc="用户选择的商品编号")
    selected_product = dspy.OutputField(desc="用户选择的商品")
    product_price = dspy.OutputField(desc="商品价格")

class ProcessPayment(dspy.Signature):
    """处理支付的签名"""
    selected_product = dspy.InputField(desc="用户选择的商品")
    payment_amount = dspy.InputField(desc="用户支付的金额")
    payment_status = dspy.OutputField(desc="支付状态")

class CheckInventory(dspy.Signature):
    """检查库存并更新的签名"""
    selected_product = dspy.InputField(desc="用户选择的商品")
    payment_status = dspy.InputField(desc="支付状态")
    inventory_info = dspy.InputField(desc="目前的商品和库存信息")
    updated_inventory = dspy.OutputField(desc="根据支付状态和用户选择的商品更新后的商品和库存信息")
    stock_status = dspy.OutputField(desc="库存更新状态")

In [15]:
# 创建模块

class VendingMachineRolePlay(dspy.Module):
    def __init__(self):
        super().__init__()
        self.inventory = {
            "饮料": {"price": 5, "stock": 10},
            "小吃": {"price": 10, "stock": 5},
            "糖果": {"price": 3, "stock": 20},
        }
        # self.initialize_inventory = dspy.ChainOfThought(InitializeInventory)
        self.select_product = dspy.ChainOfThought(SelectProduct)
        self.process_payment = dspy.ChainOfThought(ProcessPayment)
        self.check_inventory = dspy.ChainOfThought(CheckInventory)

    def forward(self, product_number=None, payment_amount=None):
        available_products = self.inventory
        
        if product_number is not None:
            selected_info = self.select_product(available_products=available_products, product_number=product_number)
            print("数据:", selected_info)
            selected_product = selected_info.selected_product
            product_price = selected_info.product_price
            
            if selected_product and available_products[selected_product]["stock"] > 0:
                if payment_amount is not None:
                    payment_result = self.process_payment(selected_product=self.inventory[selected_product], payment_amount=payment_amount)
                    print("数据2:", payment_result)
                    payment_status = payment_result.payment_status
                    
                    # 检查并更新库存
                    inventory_result = self.check_inventory(selected_product=selected_product, payment_status=payment_status, inventory_info=self.inventory)
                    print("数据3:", inventory_result)

                    # 更新本地库存
                    self.inventory = ast.literal_eval(inventory_result.updated_inventory)
                    
                    return dspy.Prediction(selected_product=selected_product, payment_status=payment_status, stock_status=inventory_result.stock_status, product_price=product_price)
                else:
                    return dspy.Prediction(selected_product=selected_product, payment_status="请支付相应金额", stock_status=None, product_price=product_price)
            else:
                return dspy.Prediction(selected_product=None, payment_status="商品缺货", stock_status=None, product_price=None)

In [16]:
# 创建数据集

products = {
    "饮料": {"price": 5, "stock": 10},
    "小吃": {"price": 10, "stock": 5},
    "糖果": {"price": 3, "stock": 20},
}

trainset_select_product = [
    dspy.Example(
        available_products=example["inputs"]["available_products"],
        product_number=example["inputs"]["product_number"],
        selected_product=example["outputs"]["selected_product"],
        product_price=example["outputs"]["product_price"]
    ).with_inputs("available_products", "product_number")
    for example in [
        {
            "inputs": {
                "available_products": products,
                "product_number": "糖果"
            },
            "outputs": {
                "selected_product": "糖果",
                "product_price": "3"
            }
        },
        {
            "inputs": {
                "available_products": products,
                "product_number": "饮料"
            },
            "outputs": {
                "selected_product": "饮料",
                "product_price": "5"
            }
        },
        {
            "inputs": {
                "available_products": products,
                "product_number": "小吃"
            },
            "outputs": {
                "selected_product": "小吃",
                "product_price": "10"
            }
        }
    ]
]

trainset_process_payment = [
    dspy.Example(
        selected_product=example["inputs"]["selected_product"],
        payment_amount=example["inputs"]["payment_amount"],
        payment_status=example["outputs"]["payment_status"]
    ).with_inputs("selected_product", "payment_amount")
    for example in [
        {
            "inputs": {
                "selected_product": products["糖果"],
                "payment_amount": "3元"
            },
            "outputs": {
                "payment_status": "支付成功"
            }
        },
        {
            "inputs": {
                "selected_product": products["饮料"],
                "payment_amount": "5元"
            },
            "outputs": {
                "payment_status": "支付成功"
            }
        },
        {
            "inputs": {
                "selected_product": products["小吃"],
                "payment_amount": "10元"
            },
            "outputs": {
                "payment_status": "支付成功"
            }
        }
    ]
]


trainset_check_inventory = [
    dspy.Example(
        selected_product=example["inputs"]["selected_product"],
        payment_status=example["inputs"]["payment_status"],
        inventory_info=example["inputs"]["inventory_info"],
        updated_inventory=example["outputs"]["updated_inventory"],
        stock_status=example["outputs"]["stock_status"]
    ).with_inputs("selected_product", "payment_status", "inventory_info")
    for example in [
        {
            "inputs": {
                "selected_product": "糖果",
                "payment_status": "支付成功",
                "inventory_info": "{'饮料': {'stock': 10}, '小吃': {'stock': 5}, '糖果': {'stock': 20}}"
            },
            "outputs": {
                "updated_inventory": "{'饮料': {'stock': 10}, '小吃': {'stock': 5}, '糖果': {'stock': 19}}",
                "stock_status": "库存更新成功"
            }
        },
        {
            "inputs": {
                "selected_product": "饮料",
                "payment_status": "支付成功",
                "inventory_info": "{'饮料': {'stock': 10}, '小吃': {'stock': 5}, '糖果': {'stock': 20}}"
            },
            "outputs": {
                "updated_inventory": "{'饮料': {'stock': 9}, '小吃': {'stock': 5}, '糖果': {'stock': 20}}",
                "stock_status": "库存更新成功"
            }
        }
    ]
]

In [17]:
# 划分数据集
from dspy.datasets import DataLoader
dl = DataLoader()

# 划分数据集
splits_select_product = dl.train_test_split(trainset_select_product, train_size=0.8)
splits_process_payment = dl.train_test_split(trainset_process_payment, train_size=0.8)
splits_check_inventory = dl.train_test_split(trainset_check_inventory, train_size=0.8)

trainset_select_product = splits_select_product['train']
devset_select_product = splits_select_product['test']
trainset_process_payment = splits_process_payment['train']
devset_process_payment = splits_process_payment['test']
trainset_check_inventory = splits_check_inventory['train']
devset_check_inventory = splits_check_inventory['test']

In [18]:
# 定义优化器 和 验证方法

# 定义优化器配置
config = dict(
    max_bootstrapped_demos=4,
    max_labeled_demos=4,
    num_candidate_programs=10,
    num_threads=4
)
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

def validate_select_product(example, pred, trace=None):
    """
    验证选择商品的输出是否正确
    """
    return (example.selected_product.lower() == pred.selected_product.lower() and
            example.product_price == pred.product_price)

def validate_process_payment(example, pred, trace=None):
    """
    验证处理支付的输出是否正确
    """
    return example.payment_status.lower() == pred.payment_status.lower()

def validate_check_inventory(example, pred, trace=None):
    """
    验证检查库存的输出是否正确
    """
    return (example.updated_inventory == pred.updated_inventory and
            example.stock_status.lower() == pred.stock_status.lower())


#创建优化器实例
optimizer_select_product = BootstrapFewShotWithRandomSearch(metric=validate_select_product, **config)
optimizer_process_payment = BootstrapFewShotWithRandomSearch(metric=validate_process_payment, **config)
optimizer_check_inventory = BootstrapFewShotWithRandomSearch(metric=validate_check_inventory, **config)

# 优化选择商品的提示
optimized_select_product = optimizer_select_product.compile(
    student=VendingMachineRolePlay().select_product,
    trainset=trainset_select_product,
    valset=devset_select_product
)

# 优化处理支付的提示
optimized_process_payment = optimizer_process_payment.compile(
    student=VendingMachineRolePlay().process_payment,
    trainset=trainset_process_payment,
    valset=devset_process_payment
)

# 优化检查库存的提示
optimized_check_inventory = optimizer_check_inventory.compile(
    student=VendingMachineRolePlay().check_inventory,
    trainset=trainset_check_inventory,
    valset=devset_check_inventory
)

Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 10 candidate sets.
Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 10 candidate sets.
Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 10 candidate sets.


Average Metric: 1 / 1  (100.0): 100%|███████████████████████████████████████████████████| 1/1 [00:00<00:00, 496.90it/s]


New best score: 100.0 for seed -3
Scores so far: [100.0]
Best score so far: 100.0


Average Metric: 1 / 1  (100.0): 100%|████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.75s/it]


Scores so far: [100.0, 100.0]
Best score so far: 100.0


...

Scores so far: [0.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
Best score so far: 100.0


100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


Bootstrapped 0 full traces after 1 examples in round 0.


Average Metric: 1 / 1  (100.0): 100%|███████████████████████████████████████████████████| 1/1 [00:00<00:00, 990.86it/s]


Scores so far: [0.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
Best score so far: 100.0
13 candidate programs found.


In [ ]:
# 主程序

# 主程序逻辑
def main():
    vending_machine = VendingMachineRolePlay()
    vending_machine.select_product = optimized_select_product
    vending_machine.process_payment = optimized_process_payment
    vending_machine.check_inventory = optimized_check_inventory
    while True:
        print("\n欢迎光临，请选择商品：")
        
        available_products = vending_machine.inventory
        available_products_info = "\n".join([f"{product} - {info['price']}元 (库存: {info['stock']})"
                                               for product, info in available_products.items()])

        print(available_products_info)
        print("输入 'q' 退出购物")

        product_number = input("请输入商品编号：")
        if product_number.lower() == 'q':
            print("感谢光临！再见！")
            break
        
        result = vending_machine(product_number=product_number)

        print("选择的商品:", result.selected_product)
        print("支付状态:", result.payment_status)
        
        if result.payment_status == "请支付相应金额":
            payment_amount = input(f"请输入支付的金额 ({result.product_price}元)：")
            payment_result = vending_machine(product_number=product_number, payment_amount=payment_amount)

            print("支付状态:", payment_result.payment_status)
            print("库存更新状态:", payment_result.stock_status)
            if payment_result.payment_status == "支付成功":
                print("请取走您的商品。谢谢光临！")
            elif payment_result.payment_status == "商品缺货":
                print("商品缺货，请选择其他商品。")
            elif payment_result.payment_status == "支付失败":
                print("支付失败，请重试。")
        print(lm.inspect_history(n=3))

if __name__ == "__main__":
    main()


欢迎光临，请选择商品：
饮料 - 5元 (库存: 10)
小吃 - 10元 (库存: 5)
糖果 - 3元 (库存: 20)
输入 'q' 退出购物


请输入商品编号： 糖果


数据: Prediction(
    reasoning="用户选择了'糖果'，这是根据提供的可用商品列表中编号为'糖果'的商品进行选择的。",
    selected_product='糖果',
    product_price='3'
)
选择的商品: 糖果
支付状态: 请支付相应金额


请输入支付的金额 (3元)： 3元


数据: Prediction(
    reasoning="用户选择了'糖果'，这是根据提供的可用商品列表中编号为'糖果'的商品进行选择的。",
    selected_product='糖果',
    product_price='3'
)
数据2: Prediction(
    reasoning='Not supplied for this particular example.',
    payment_status='支付成功'
)
数据3: Prediction(
    reasoning='用户支付成功购买了糖果，库存信息需要更新。',
    updated_inventory="{'饮料': {'price': 5, 'stock': 10}, '小吃': {'price': 10, 'stock': 5}, '糖果': {'price': 3, 'stock': 19}}",
    stock_status='库存更新成功'
)
支付状态: 支付成功
库存更新状态: 库存更新成功
请取走您的商品。谢谢光临！




System message:

Your input fields are:
1. `available_products` (str): 可用商品列表
2. `product_number` (str): 用户选择的商品编号

Your output fields are:
1. `reasoning` (str)
2. `selected_product` (str): 用户选择的商品
3. `product_price` (str): 商品价格

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## available_products ## ]]
{available_products}

[[ ## product_number ## ]]
{product_number}

[[ ## reasoning ## ]]
{reasoning}

[[ ## selected_product ## ]]
{selected_product}

[[ ## pr